In [65]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import os
import datetime as dt
import xgboost as xgb
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import catboost
import sklearn
from sklearn.model_selection import cross_val_score, KFold, learning_curve, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import operator
import matplotlib.pyplot as plt
import xgboost as xgb
import os
import seaborn as sns
import datetime
from time import strftime, gmtime, time
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

tqdm.pandas()

In [2]:
sns.set_context("poster") 
plt.style.use('seaborn-poster')

In [3]:
# !pip install xgboost
# !pip install sklearn

In [37]:
def feature_importances(model) -> pd.DataFrame:

    importance_dict = {}

    for feature, importance in zip(X_train.columns, model.feature_importances_):
        importance_dict[feature] = importance

    df_importance = pd.DataFrame(sorted(importance_dict.items(), key=operator.itemgetter(1)))
    df_importance = df_importance.set_index(0).sort_values(by=[1], ascending=False)

    return df_importance

In [79]:
cat_tester = pd.read_csv("Merged_algo_tester.csv")
cat_tester = cat_tester.drop(columns= 'Unnamed: 0')
cat_tester.columns

Index(['Location', 'Visits', 'Date', 'Time', 'Windspeed', 'Temperature',
       'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
       'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
       'Holiday_name', 'Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday'],
      dtype='object')

In [32]:
X, y = cat_tester.loc[:, cat_tester.columns != "Visits"], cat_tester[["Visits"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
cat_features = ['Date','Time','Location']

X_train = X_train[['Location', 'Date', 'Time', 'Windspeed', 'Temperature',
       'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
       'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
       'Holiday_name', 'Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
X_test = X_test[['Location', 'Date', 'Time', 'Windspeed', 'Temperature',
       'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
       'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
       'Holiday_name', 'Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]

assert "ARRIVAL_DELAY" not in X_train.columns
assert y_train.shape[1] == 1

In [73]:
cat_scores = {}

for iterations in range(25, 2000, 50):
    
    print(f"Fitting model for {iterations} iterations...")
    
    start_time = time()
    
    cat_scores[iterations] = {}
    
    cat_model = catboost.CatBoostRegressor(iterations=iterations, cat_features=cat_features, verbose=False)
    
    cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))
     
    
    cat_scores[iterations]["RMSE train"] = cat_model.get_best_score()["learn"]["RMSE"]  
    pred = cat_model.predict(X_train)
    mae_train = mean_absolute_error(y_train, pred) 
    cat_scores[iterations]['MAE Train'] = mae_train
    
    cat_scores[iterations]["RMSE test"] = cat_model.get_best_score()["validation"]["RMSE"]
    pred = cat_model.predict(X_test)
    mae_val = mean_absolute_error(y_test, pred)
    cat_scores[iterations]['MAE Test'] = mae_val
    
    cat_scores[iterations]["time"] = time() - start_time
    
    print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")

Fitting model for 25 iterations...


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Fitting model for 25 iterations took 0.9398519992828369 seconds...
Fitting model for 75 iterations...
Fitting model for 75 iterations took 2.0904860496520996 seconds...
Fitting model for 125 iterations...
Fitting model for 125 iterations took 3.1484148502349854 seconds...
Fitting model for 175 iterations...
Fitting model for 175 iterations took 3.84513521194458 seconds...
Fitting model for 225 iterations...
Fitting model for 225 iterations took 9.788969993591309 seconds...
Fitting model for 275 iterations...
Fitting model for 275 iterations took 11.950329065322876 seconds...
Fitting model for 325 iterations...
Fitting model for 325 iterations took 14.838788032531738 seconds...
Fitting model for 375 iterations...
Fitting model for 375 iterations took 18.764678239822388 seconds...
Fitting model for 425 iterations...
Fitting model for 425 iterations took 20.87704110145569 seconds...
Fitting model for 475 iterations...
Fitting model for 475 iterations took 23.40273928642273 seconds...
Fitt

In [74]:
cat_scores

{25: {'RMSE train': 164.38554879271607,
  'MAE Train': 96.78067384866524,
  'RMSE test': 160.61147973062833,
  'MAE Test': 97.69007796033908,
  'time': 0.9398510456085205},
 75: {'RMSE train': 156.46463096906461,
  'MAE Train': 91.21572106849416,
  'RMSE test': 153.0645880352667,
  'MAE Test': 92.44874016939207,
  'time': 2.0904839038848877},
 125: {'RMSE train': 153.30503726757127,
  'MAE Train': 89.25088785729297,
  'RMSE test': 150.78110065745108,
  'MAE Test': 90.67920822463255,
  'time': 3.148413896560669},
 175: {'RMSE train': 152.12440556490324,
  'MAE Train': 88.56739910474892,
  'RMSE test': 149.85730174386347,
  'MAE Test': 90.29616728324925,
  'time': 3.8451340198516846},
 225: {'RMSE train': 140.81644413027533,
  'MAE Train': 79.80359309331942,
  'RMSE test': 140.4361750971589,
  'MAE Test': 82.38038945276348,
  'time': 9.788968086242676},
 275: {'RMSE train': 140.19577242613715,
  'MAE Train': 79.27045897571035,
  'RMSE test': 139.90651602156174,
  'MAE Test': 81.957816714

In [75]:
cat_model.get_best_score()

{'learn': {'RMSE': 130.50255867770392},
 'validation': {'RMSE': 133.16293884290465}}

In [77]:
feature_importances(cat_model)

,1
0,
Time,25.121006
Location,23.784089
Date,22.008551
Year,8.230050
Month,4.711153
Temperature,4.416355
Day,3.235363
Windspeed,2.338139
Sun duration,1.713165
